In [1]:
import os
import sys
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.model_selection import train_test_split
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, confusion_matrix
from tabulate import tabulate
from autogluon.tabular import TabularDataset, TabularPredictor
from imblearn.over_sampling import SMOTE, SMOTENC, RandomOverSampler
from imblearn.under_sampling import RandomUnderSampler

pd.set_option('display.max_columns', None)

c:\Users\olaya\anaconda3\envs\superconductor\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
# LEO LOS DATOS DE DOS

dos_csv_path = r"D:\tfg\data\dos_data.csv"
data_materials = pd.read_csv(dos_csv_path)

data_materials['ICSD'] = pd.to_numeric(data_materials['ICSD'], errors='coerce')

num_nulos = data_materials['ICSD'].isnull().sum()
len_df_pre = len(data_materials)
data_materials.dropna(subset=['ICSD'], inplace=True)
print(f"{num_nulos} ({num_nulos/len_df_pre*100:.4f} %) rows where dropped because of null on ICSD.")

data_materials.ICSD = data_materials.ICSD.astype('Int64')

data_materials

C:\Users\olaya\AppData\Local\Temp\ipykernel_30312\2015048428.py:4: DtypeWarning: Columns (2) have mixed types. Specify dtype option on import or set low_memory=False.
  data_materials = pd.read_csv(dos_csv_path)


1 (0.0017 %) rows where dropped because of null on ICSD.


bravais_lattice material_name    ICSD  fermi_energy  magnetic  DOS_0  \
0                 BCC      Ag1F6Sb1   28676      -2.03066     False    0.0   
1                 BCC      Ag1F6Sb1  411795      -2.03253     False    0.0   
2                 BCC        Ag1Te3   37186       5.57824     False    0.0   
3                 BCC     Ag3Au1Se2   15734       2.21332     False    0.0   
4                 BCC     Ag3Au1Se2  171959       2.22832     False    0.0   
...               ...           ...     ...           ...       ...    ...   
60212             TRI     Se4Tl4Zr1  261209       3.16038     False    0.0   
60213             TRI     Se6Si2Tl4   35042       2.64747     False    0.0   
60214             TRI    Se11Ta2Tl4  412581       2.44024     False    0.0   
60215             TRI     Si2Te6Tl6  416310       4.07369     False    0.0   
60216             TRI        Tl3Yb8  104202       2.27586     False    0.0   

       DOS_1  DOS_2  DOS_3  DOS_4  DOS_5  DOS_6  DOS_7  DOS_8  DOS_9  DOS_10  \
0        0.0    0.0    0.0    0.0    0.0    0.0    0.0    0.0    0.0     0.0   
1        0.0    0.0    0.0    0.0    0.0    0.0    0.0    0.0    0.0     0.0   
2        0.0    0.0    0.0    0.0    0.0    0.0    0.0    0.0    0.0     0.0   
3        0.0    0.0    0.0    0.0    0.0    0.0    0.0    0.0    0.0     0.0   
4        0.0    0.0    0.0    0.0    0.0    0.0    0.0    0.0    0.0     0.0   
...      ...    ...    ...    ...    ...    ...    ...    ...    ...     ...   
60212    0.0    0.0    0.0    0.0    0.0    0.0    0.0    0.0    0.0     0.0   
60213    0.0    0.0    0.0    0.0    0.0    0.0    0.0    0.0    0.0     0.0   
60214    0.0    0.0    0.0    0.0    0.0    0.0    0.0    0.0    0.0     0.0   
60215    0.0    0.0    0.0    0.0    0.0    0.0    0.0    0.0    0.0     0.0   
60216    0.0    0.0    0.0    0.0    0.0    0.0    0.0    0.0    0.0     0.0   

       DOS_11  DOS_12  DOS_13  DOS_14  DOS_15  DOS_16  DOS_17  DOS_18  DOS_19  \
0         0.0     0.0     0.0     0.0     0.0     0.0     0.0     0.0     0.0   
1         0.0     0.0     0.0     0.0     0.0     0.0     0.0     0.0     0.0   
2         0.0     0.0     0.0     0.0     0.0     0.0     0.0     0.0     0.0   
3         0.0     0.0     0.0     0.0     0.0     0.0     0.0     0.0     0.0   
4         0.0     0.0     0.0     0.0     0.0     0.0     0.0     0.0     0.0   
...       ...     ...     ...     ...     ...     ...     ...     ...     ...   
60212     0.0     0.0     0.0     0.0     0.0     0.0     0.0     0.0     0.0   
60213     0.0     0.0     0.0     0.0     0.0     0.0     0.0     0.0     0.0   
60214     0.0     0.0     0.0     0.0     0.0     0.0     0.0     0.0     0.0   
60215     0.0     0.0     0.0     0.0     0.0     0.0     0.0     0.0     0.0   
60216     0.0     0.0     0.0     0.0     0.0     0.0     0.0     0.0     0.0   

       DOS_20  DOS_21  DOS_22  DOS_23  DOS_24  DOS_25  DOS_26  DOS_27  DOS_28  \
0         0.0     0.0     0.0     0.0     0.0     0.0     0.0     0.0     0.0   
1         0.0     0.0     0.0     0.0     0.0     0.0     0.0     0.0     0.0   
2         0.0     0.0     0.0     0.0     0.0     0.0     0.0     0.0     0.0   
3         0.0     0.0     0.0     0.0     0.0     0.0     0.0     0.0     0.0   
4         0.0     0.0     0.0     0.0     0.0     0.0     0.0     0.0     0.0   
...       ...     ...     ...     ...     ...     ...     ...     ...     ...   
60212     0.0     0.0     0.0     0.0     0.0     0.0     0.0     0.0     0.0   
60213     0.0     0.0     0.0     0.0     0.0     0.0     0.0     0.0     0.0   
60214     0.0     0.0     0.0     0.0     0.0     0.0     0.0     0.0     0.0   
60215     0.0     0.0     0.0     0.0     0.0     0.0     0.0     0.0     0.0   
60216     0.0     0.0     0.0     0.0     0.0     0.0     0.0     0.0     0.0   

       DOS_29  DOS_30  DOS_31  DOS_32  DOS_33  DOS_34  DOS_35  DOS_36  DOS_37  \
0         0.0     0.0     0.0     0.0     0.0     0.0     0.0     0.0     0.0   


In [3]:
# LEO DAOS DE SUPERCONDUCTORES
data_path = r"D:\tfg\data\3DSC"
file = '3DSC_ICSD_only_IDs.csv'
file_path = os.path.join(data_path, file)

supercon_data = pd.read_csv(file_path, skiprows=1)
supercon_data.columns = ['chemical_formula', 'critical_temperature_k', 'ICSD', 'synth_doped']
supercon_data['ICSD'] = supercon_data['ICSD'].str.replace('ICSD-', '')

# Superconductores que aparecen en 3DSC
ICSD_supercon = supercon_data.ICSD.unique().astype(int).tolist()

In [4]:
# AÑADO VARIABLE PARA VER CUALES SON SUPERCON
data_materials.insert(4, 'is_superconductor', data_materials.ICSD.isin(ICSD_supercon))

value_counts = data_materials['is_superconductor'].value_counts()

total_count = len(data_materials['is_superconductor'])
true_count = value_counts.get(True, 0)  
false_count = value_counts.get(False, 0)
true_percentage = (true_count / total_count) * 100
print(f"Percentage of superconductors: {true_percentage:.2f}%")

data_materials = data_materials.rename(columns={"magnetic": "is_magnetic"})
data_materials[(data_materials.is_superconductor!='')&(data_materials.bravais_lattice=='BCC')&(data_materials.ICSD>500000)].head(5)
# data_materials.head(20)

Percentage of superconductors: 8.17%


bravais_lattice material_name    ICSD  fermi_energy  is_superconductor  \
8              BCC     Ag3Au1Te2  604514       3.59922              False   
9              BCC     Ag3Au1Te2  604788       3.61422              False   
12             BCC        Ag5Cd8  604897       2.85317               True   
76             BCC   Al2Dy3H9Ni6  657447       5.95833              False   
78             BCC     Al2Dy3Ni6  607315       5.96291              False   

    is_magnetic  DOS_0  DOS_1  DOS_2  DOS_3  DOS_4  DOS_5  DOS_6  DOS_7  \
8         False    0.0    0.0    0.0    0.0    0.0    0.0    0.0    0.0   
9         False    0.0    0.0    0.0    0.0    0.0    0.0    0.0    0.0   
12        False    0.0    0.0    0.0    0.0    0.0    0.0    0.0    0.0   
76        False    0.0    0.0    0.0    0.0    0.0    0.0    0.0    0.0   
78        False    0.0    0.0    0.0    0.0    0.0    0.0    0.0    0.0   

    DOS_8  DOS_9  DOS_10  DOS_11  DOS_12  DOS_13  DOS_14  DOS_15  DOS_16  \
8     0.0    0.0     0.0     0.0     0.0     0.0     0.0     0.0     0.0   
9     0.0    0.0     0.0     0.0     0.0     0.0     0.0     0.0     0.0   
12    0.0    0.0     0.0     0.0     0.0     0.0     0.0     0.0     0.0   
76    0.0    0.0     0.0     0.0     0.0     0.0     0.0     0.0     0.0   
78    0.0    0.0     0.0     0.0     0.0     0.0     0.0     0.0     0.0   

    DOS_17  DOS_18  DOS_19  DOS_20  DOS_21  DOS_22  DOS_23  DOS_24  DOS_25  \
8      0.0     0.0     0.0     0.0     0.0     0.0     0.0     0.0     0.0   
9      0.0     0.0     0.0     0.0     0.0     0.0     0.0     0.0     0.0   
12     0.0     0.0     0.0     0.0     0.0     0.0     0.0     0.0     0.0   
76     0.0     0.0     0.0     0.0     0.0     0.0     0.0     0.0     0.0   
78     0.0     0.0     0.0     0.0     0.0     0.0     0.0     0.0     0.0   

    DOS_26  DOS_27  DOS_28  DOS_29  DOS_30  DOS_31  DOS_32  DOS_33  DOS_34  \
8      0.0     0.0     0.0     0.0     0.0     0.0     0.0     0.0     0.0   
9      0.0     0.0     0.0     0.0     0.0     0.0     0.0     0.0     0.0   
12     0.0     0.0     0.0     0.0     0.0     0.0     0.0     0.0     0.0   
76     0.0     0.0     0.0     0.0     0.0     0.0     0.0     0.0     0.0   
78     0.0     0.0     0.0     0.0     0.0     0.0     0.0     0.0     0.0   

    DOS_35  DOS_36  DOS_37  DOS_38  DOS_39  DOS_40  DOS_41  DOS_42  DOS_43  \
8      0.0     0.0     0.0     0.0     0.0     0.0     0.0     0.0     0.0   
9      0.0     0.0     0.0     0.0     0.0     0.0     0.0     0.0     0.0   
12     0.0     0.0     0.0     0.0     0.0     0.0     0.0     0.0     0.0   
76     0.0     0.0     0.0     0.0     0.0     0.0     0.0     0.0     0.0   
78     0.0     0.0     0.0     0.0     0.0     0.0     0.0     0.0     0.0   

    DOS_44  DOS_45  DOS_46  DOS_47  DOS_48  DOS_49  DOS_50  DOS_51  DOS_52  \
8      0.0     0.0     0.0     0.0     0.0     0.0     0.0     0.0     0.0   
9      0.0     0.0     0.0     0.0     0.0     0.0     0.0     0.0     0.0   
12     0.0     0.0     0.0     0.0     0.0     0.0     0.0     0.0     0.0   
76     0.0     0.0     0.0     0.0     0.0     0.0     0.0     0.0     0.0   
78     0.0     0.0     0.0     0.0     0.0     0.0     0.0     0.0     0.0   

    DOS_53  DOS_54  DOS_55  DOS_56  DOS_57  DOS_58  DOS_59  DOS_60  DOS_61  \
8      0.0     0.0     0.0     0.0     0.0     0.0     0.0     0.0     0.0   
9      0.0     0.0     0.0     0.0     0.0     0.0     0.0     0.0     0.0   
12     0.0     0.0     0.0     0.0     0.0     0.0     0.0     0.0     0.0   
76     0.0     0.0     0.0     0.0     0.0     0.0     0.0     0.0     0.0   
78     0.0     0.0     0.0     0.0     0.0     0.0     0.0     0.0     0.0   

    DOS_62  DOS_63  DOS_64  DOS_65  DOS_66  DOS_67  DOS_68  DOS_69  DOS_70  \
8      0.0     0.0     0.0     0.0     0.0     0.0     0.0     0.0     0.0   
9      0.0     0.0     0.0     0.0     0.0     0.0     0.0     0.0     0.0   
12     0.0     0.0     0.0     0.0     0.0     0.0   

In [5]:
# Columnas por tipo de dato
categorical_cols = ['bravais_lattice']
numerical_cols = ['fermi_energy'] + [f'DOS_{i}' for i in range(2000)]
target_col = ['is_superconductor']

# Transformaciones
categorical_transformer = OneHotEncoder(handle_unknown='ignore')
numerical_transformer = StandardScaler()

preprocessor = ColumnTransformer(
    transformers=[
        ('cat', categorical_transformer, categorical_cols),
        ('num', numerical_transformer, numerical_cols)
    ])

# Preprocesamiento y división
X = data_materials.drop(target_col, axis=1)
X = X.fillna(0)
y = data_materials[target_col]
y = y.iloc[:, 0]
X_processed = preprocessor.fit_transform(X)
X_train, X_test, y_train, y_test = train_test_split(X_processed, y, test_size=0.2, random_state=42)

In [16]:
# undersampling train

rus = RandomUnderSampler(random_state=42) 
X_under_resampled, y_under_resampled = rus.fit_resample(X_train, y_train)

In [19]:
train_data_random_underresampled = pd.concat([pd.DataFrame(X_under_resampled), pd.DataFrame(y_under_resampled)], axis=1)
test_data = pd.concat([pd.DataFrame(X_test).reset_index(drop=True), pd.DataFrame(y_test).reset_index(drop=True)], axis=1)

In [12]:
path_for_run = r'AutogluonModel/RandomUnderSampler'

predictor = TabularPredictor(label='is_superconductor', problem_type='binary', eval_metric='f1', path = path_for_run).fit(
        TabularDataset(train_data_random_underresampled),
        # num_bag_folds=5, num_bag_sets=1, num_stack_levels=3,
        # hyperparameters=hyperparameters,
        # presets='best_quality'
        )

No presets specified! To achieve strong results with AutoGluon, it is recommended to use the available presets.
	Recommended Presets (For more details refer to https://auto.gluon.ai/stable/tutorials/tabular/tabular-essentials.html#presets):
	presets='best_quality'   : Maximize accuracy. Default time_limit=3600.
	presets='high_quality'   : Strong accuracy with fast inference speed. Default time_limit=3600.
	presets='good_quality'   : Good accuracy with very fast inference speed. Default time_limit=3600.
	presets='medium_quality' : Fast training time, ideal for initial prototyping.
Cannot interpret 'BooleanDtype' as a data type
	Consider setting `time_limit` to ensure training finishes within an expected duration or experiment with a small portion of `train_data` to identify an ideal `presets` and `hyperparameters` configuration.
Beginning AutoGluon training ...
AutoGluon will save models to "AutogluonModel/RandomUnderSampler"
=================== System Info ===================
AutoGluon

[1000]	valid_set's binary_logloss: 0.49896	valid_set's f1: 0.813028


	0.817	 = Validation score   (f1)
	18.35s	 = Training   runtime
	0.02s	 = Validation runtime
Fitting model: LightGBM ...
	0.8132	 = Validation score   (f1)
	19.41s	 = Training   runtime
	0.02s	 = Validation runtime
Fitting model: RandomForestGini ...
	0.7865	 = Validation score   (f1)
	3.97s	 = Training   runtime
	0.04s	 = Validation runtime
Fitting model: RandomForestEntr ...
	0.7899	 = Validation score   (f1)
	4.28s	 = Training   runtime
	0.04s	 = Validation runtime
Fitting model: CatBoost ...
	0.8126	 = Validation score   (f1)
	106.26s	 = Training   runtime
	0.02s	 = Validation runtime
Fitting model: ExtraTreesGini ...
	0.7923	 = Validation score   (f1)
	3.46s	 = Training   runtime
	0.04s	 = Validation runtime
Fitting model: ExtraTreesEntr ...
	0.7957	 = Validation score   (f1)
	3.54s	 = Training   runtime
	0.04s	 = Validation runtime
Fitting model: NeuralNetFastAI ...
	0.7853	 = Validation score   (f1)
	13.01s	 = Training   runtime
	0.06s	 = Validation runtime
Fitting model: XGBoos

In [21]:
predictor.evaluate(test_data)

{'f1': 0.3569919729615547,
 'accuracy': 0.7472600464961807,
 'balanced_accuracy': 0.8096855056844596,
 'mcc': 0.36085086656484083,
 'roc_auc': 0.8934162870486092,
 'precision': 0.22366331392271044,
 'recall': 0.8838912133891214}

In [22]:
predictor.leaderboard(test_data, extra_metrics=['accuracy', 'roc_auc', 'precision', 'recall'], silent=True)

,model,score_test,accuracy,roc_auc,precision,recall,score_val,eval_metric,pred_time_test,pred_time_val,fit_time,pred_time_test_marginal,pred_time_val_marginal,fit_time_marginal,stack_level,can_infer,fit_order
0,LightGBM,0.366956,0.763949,0.889908,0.233098,0.861925,0.813187,f1,0.168499,0.022500,19.414000,0.168499,0.022500,19.414000,1,True,4
1,ExtraTreesGini,0.361129,0.753819,0.887131,0.227408,0.876569,0.792317,f1,0.192999,0.038000,3.462500,0.192999,0.038000,3.462500,1,True,8
2,RandomForestGini,0.361033,0.755480,0.878983,0.227758,0.870293,0.786490,f1,0.199501,0.035501,3.969005,0.199501,0.035501,3.969005,1,True,5
3,ExtraTreesEntr,0.357985,0.751328,0.887279,0.225128,0.873431,0.795673,f1,0.197001,0.042000,3.537001,0.197001,0.042000,3.537001,1,True,9
4,WeightedEnsemble_L2,0.356992,0.747260,0.893416,0.223663,0.883891,0.829152,f1,4.582620,0.769960,242.020300,0.003000,0.000499,0.244099,2,True,14
5,XGBoost,0.354923,0.755231,0.884348,0.224405,0.848326,0.812950,f1,0.811500,0.069000,34.113705,0.811500,0.069000,34.113705,1,True,11
6,LightGBMXT,0.354276,0.754234,0.887269,0.223815,0.849372,0.816970,f1,0.182000,0.023500,18.353570,0.182000,0.023500,18.353570,1,True,3
7,RandomForestEntr,0.353589,0.746513,0.879465,0.221662,0.873431,0.789916,f1,0.174477,0.035501,4.279501,0.174477,0.035501,4.279501,1,True,6
8,LightGBMLarge,0.353067,0.748672,0.886198,0.221864,0.864017,0.814545,f1,0.152423,0.021499,30.427959,0.152423,0.021499,30.427959,1,True,13
9,CatBoost,0.349623,0.749170,0.885090,0.220114,0.849372,0.812576,f1,0.062000,0.021499,106.255074,0.062000,0.021499,106.255074,1,True,7


In [6]:
# Oversampling train
ros = RandomOverSampler(random_state=42)  
X_over_resampled, y_over_resampled = ros.fit_resample(X_train, y_train)

In [7]:
train_data_random_overresampled = pd.concat([pd.DataFrame(X_over_resampled), pd.DataFrame(y_over_resampled)], axis=1)
test_data = pd.concat([pd.DataFrame(X_test).reset_index(drop=True), pd.DataFrame(y_test).reset_index(drop=True)], axis=1)

In [10]:
path_for_run = r'AutogluonModel/RandomOverSampler'

predictor2 = TabularPredictor(label='is_superconductor', problem_type='binary', eval_metric='f1', path = path_for_run).fit(
        TabularDataset(train_data_random_overresampled),
        ag_args_fit={"ag.max_memory_usage_ratio": 1.61}
        # num_bag_folds=5, num_bag_sets=1, num_stack_levels=3,
        # hyperparameters=hyperparameters,
        # presets='best_quality'
        )

No presets specified! To achieve strong results with AutoGluon, it is recommended to use the available presets.
	Recommended Presets (For more details refer to https://auto.gluon.ai/stable/tutorials/tabular/tabular-essentials.html#presets):
	presets='best_quality'   : Maximize accuracy. Default time_limit=3600.
	presets='high_quality'   : Strong accuracy with fast inference speed. Default time_limit=3600.
	presets='good_quality'   : Good accuracy with very fast inference speed. Default time_limit=3600.
	presets='medium_quality' : Fast training time, ideal for initial prototyping.
Cannot interpret 'BooleanDtype' as a data type
	Consider setting `time_limit` to ensure training finishes within an expected duration or experiment with a small portion of `train_data` to identify an ideal `presets` and `hyperparameters` configuration.
Beginning AutoGluon training ...
AutoGluon will save models to "AutogluonModel/RandomOverSampler"
=================== System Info ===================
AutoGluon 

[1000]	valid_set's binary_logloss: 0.254142	valid_set's f1: 0.886646
[2000]	valid_set's binary_logloss: 0.215518	valid_set's f1: 0.903652
[3000]	valid_set's binary_logloss: 0.198284	valid_set's f1: 0.9125


	0.913	 = Validation score   (f1)
	184.13s	 = Training   runtime
	0.06s	 = Validation runtime
Fitting model: LightGBM ...


[1000]	valid_set's binary_logloss: 0.206303	valid_set's f1: 0.912335
[2000]	valid_set's binary_logloss: 0.164603	valid_set's f1: 0.933801
[3000]	valid_set's binary_logloss: 0.145459	valid_set's f1: 0.940856
[4000]	valid_set's binary_logloss: 0.139958	valid_set's f1: 0.942957


	0.9436	 = Validation score   (f1)
	207.27s	 = Training   runtime
	0.07s	 = Validation runtime
Fitting model: RandomForestGini ...
	0.9835	 = Validation score   (f1)
	64.88s	 = Training   runtime
	0.06s	 = Validation runtime
Fitting model: RandomForestEntr ...
	0.9843	 = Validation score   (f1)
	65.0s	 = Training   runtime
	0.06s	 = Validation runtime
Fitting model: CatBoost ...
	0.9401	 = Validation score   (f1)
	908.22s	 = Training   runtime
	0.03s	 = Validation runtime
Fitting model: ExtraTreesGini ...
	0.9823	 = Validation score   (f1)
	71.33s	 = Training   runtime
	0.06s	 = Validation runtime
Fitting model: ExtraTreesEntr ...
	0.9827	 = Validation score   (f1)
	70.4s	 = Training   runtime
	0.06s	 = Validation runtime
Fitting model: NeuralNetFastAI ...
	0.8556	 = Validation score   (f1)
	147.64s	 = Training   runtime
	0.11s	 = Validation runtime
Fitting model: XGBoost ...
	0.9483	 = Validation score   (f1)
	1433.8s	 = Training   runtime
	0.25s	 = Validation runtime
Fitting model: N

[1000]	valid_set's binary_logloss: 0.146168	valid_set's f1: 0.947123


	0.9479	 = Validation score   (f1)
	233.39s	 = Training   runtime
	0.05s	 = Validation runtime
Fitting model: WeightedEnsemble_L2 ...
	Ensemble Weights: {'RandomForestEntr': 0.571, 'ExtraTreesGini': 0.286, 'LightGBM': 0.143}
	0.985	 = Validation score   (f1)
	0.35s	 = Training   runtime
	0.0s	 = Validation runtime
AutoGluon training complete, total runtime = 3996.07s ... Best model: "WeightedEnsemble_L2"
TabularPredictor saved. To load, use: predictor = TabularPredictor.load("AutogluonModel/RandomOverSampler")


In [11]:
predictor2.evaluate(test_data)

{'f1': 0.5539295392953929,
 'accuracy': 0.931667220192627,
 'balanced_accuracy': 0.7502139596804869,
 'mcc': 0.5173821228046586,
 'roc_auc': 0.9019167976084816,
 'precision': 0.5748031496062992,
 'recall': 0.5345188284518828}

In [12]:
predictor2.leaderboard(test_data, extra_metrics=['accuracy', 'roc_auc', 'precision', 'recall'], silent=True)

,model,score_test,accuracy,roc_auc,precision,recall,score_val,eval_metric,pred_time_test,pred_time_val,fit_time,pred_time_test_marginal,pred_time_val_marginal,fit_time_marginal,stack_level,can_infer,fit_order
0,RandomForestGini,0.555014,0.931833,0.867950,0.575928,0.535565,0.983478,f1,0.351501,0.057500,64.877023,0.351501,0.057500,64.877023,1,True,5
1,WeightedEnsemble_L2,0.553930,0.931667,0.901917,0.574803,0.534519,0.985028,f1,1.042500,0.188500,343.937511,0.004999,0.001500,0.349319,2,True,14
2,RandomForestEntr,0.552890,0.931916,0.870791,0.577449,0.530335,0.984252,f1,0.308500,0.060000,64.995672,0.308500,0.060000,64.995672,1,True,6
3,ExtraTreesEntr,0.552239,0.930256,0.876716,0.563043,0.541841,0.982704,f1,0.455998,0.060374,70.399821,0.455998,0.060374,70.399821,1,True,9
4,ExtraTreesGini,0.549545,0.930173,0.877159,0.563117,0.536611,0.982318,f1,0.381501,0.061000,71.325867,0.381501,0.061000,71.325867,1,True,8
5,CatBoost,0.484704,0.892312,0.895116,0.390775,0.638075,0.940078,f1,0.152500,0.028499,908.221744,0.152500,0.028499,908.221744,1,True,7
6,LightGBM,0.471009,0.889406,0.883091,0.379641,0.620293,0.943602,f1,0.347500,0.066000,207.266653,0.347500,0.066000,207.266653,1,True,4
7,XGBoost,0.469420,0.889073,0.882152,0.378361,0.618201,0.948309,f1,1.779500,0.245499,1433.802356,1.779500,0.245499,1433.802356,1,True,11
8,LightGBMLarge,0.465600,0.889073,0.889044,0.376943,0.608787,0.947860,f1,0.325999,0.054499,233.387927,0.325999,0.054499,233.387927,1,True,13
9,LightGBMXT,0.448696,0.868399,0.889015,0.336113,0.674686,0.912993,f1,0.401999,0.058498,184.127521,0.401999,0.058498,184.127521,1,True,3


In [13]:
# SIN HACER NADA

train_data = pd.concat([pd.DataFrame(X_train).reset_index(drop=True), pd.DataFrame(y_train).reset_index(drop=True)], axis=1)
test_data = pd.concat([pd.DataFrame(X_test).reset_index(drop=True), pd.DataFrame(y_test).reset_index(drop=True)], axis=1)

In [14]:
path_for_run = r'AutogluonModel/Raw'

predictor3 = TabularPredictor(label='is_superconductor', problem_type='binary', eval_metric='f1', path = path_for_run).fit(
        TabularDataset(train_data),
        ag_args_fit={"ag.max_memory_usage_ratio": 1.61}
        # num_bag_folds=5, num_bag_sets=1, num_stack_levels=3,
        # hyperparameters=hyperparameters,
        # presets='best_quality'
        )

No presets specified! To achieve strong results with AutoGluon, it is recommended to use the available presets.
	Recommended Presets (For more details refer to https://auto.gluon.ai/stable/tutorials/tabular/tabular-essentials.html#presets):
	presets='best_quality'   : Maximize accuracy. Default time_limit=3600.
	presets='high_quality'   : Strong accuracy with fast inference speed. Default time_limit=3600.
	presets='good_quality'   : Good accuracy with very fast inference speed. Default time_limit=3600.
	presets='medium_quality' : Fast training time, ideal for initial prototyping.
Cannot interpret 'BooleanDtype' as a data type
	Consider setting `time_limit` to ensure training finishes within an expected duration or experiment with a small portion of `train_data` to identify an ideal `presets` and `hyperparameters` configuration.
Beginning AutoGluon training ...
AutoGluon will save models to "AutogluonModel/Raw"
=================== System Info ===================
AutoGluon Version:  1.1.

[1000]	valid_set's binary_logloss: 0.186479	valid_set's f1: 0.489933
[2000]	valid_set's binary_logloss: 0.194007	valid_set's f1: 0.511327
[3000]	valid_set's binary_logloss: 0.204383	valid_set's f1: 0.530351
[4000]	valid_set's binary_logloss: 0.21512	valid_set's f1: 0.540881
[5000]	valid_set's binary_logloss: 0.225196	valid_set's f1: 0.544892
[6000]	valid_set's binary_logloss: 0.236179	valid_set's f1: 0.541538


	0.5538	 = Validation score   (f1)
	223.72s	 = Training   runtime
	0.09s	 = Validation runtime
Fitting model: LightGBM ...


[1000]	valid_set's binary_logloss: 0.186206	valid_set's f1: 0.488599
[2000]	valid_set's binary_logloss: 0.20976	valid_set's f1: 0.52795


	0.5342	 = Validation score   (f1)
	109.11s	 = Training   runtime
	0.07s	 = Validation runtime
Fitting model: RandomForestGini ...
	0.5275	 = Validation score   (f1)
	35.63s	 = Training   runtime
	0.06s	 = Validation runtime
Fitting model: RandomForestEntr ...
	0.5315	 = Validation score   (f1)
	29.73s	 = Training   runtime
	0.06s	 = Validation runtime
Fitting model: CatBoost ...
	0.4951	 = Validation score   (f1)
	473.56s	 = Training   runtime
	0.03s	 = Validation runtime
Fitting model: ExtraTreesGini ...
	0.5413	 = Validation score   (f1)
	29.51s	 = Training   runtime
	0.06s	 = Validation runtime
Fitting model: ExtraTreesEntr ...
	0.5387	 = Validation score   (f1)
	29.47s	 = Training   runtime
	0.06s	 = Validation runtime
Fitting model: NeuralNetFastAI ...
	0.3617	 = Validation score   (f1)
	78.27s	 = Training   runtime
	0.11s	 = Validation runtime
Fitting model: XGBoost ...
	0.527	 = Validation score   (f1)
	168.47s	 = Training   runtime
	0.17s	 = Validation runtime
Fitting model: N

In [15]:
predictor3.evaluate(test_data)

{'f1': 0.528127068166777,
 'accuracy': 0.9408003985386915,
 'balanced_accuracy': 0.7016473763335688,
 'mcc': 0.5199846457542437,
 'roc_auc': 0.9057275063093578,
 'precision': 0.7189189189189189,
 'recall': 0.41736401673640167}

In [16]:
predictor3.leaderboard(test_data, extra_metrics=['accuracy', 'roc_auc', 'precision', 'recall'], silent=True)

,model,score_test,accuracy,roc_auc,precision,recall,score_val,eval_metric,pred_time_test,pred_time_val,fit_time,pred_time_test_marginal,pred_time_val_marginal,fit_time_marginal,stack_level,can_infer,fit_order
0,ExtraTreesGini,0.531799,0.935819,0.872035,0.631655,0.459205,0.541311,f1,0.274501,0.056499,29.508501,0.274501,0.056499,29.508501,1,True,8
1,WeightedEnsemble_L2,0.528127,0.940800,0.905728,0.718919,0.417364,0.592145,f1,14.286394,3.275064,830.421657,0.003001,0.001000,0.294000,2,True,14
2,ExtraTreesEntr,0.524729,0.934573,0.873938,0.619658,0.455021,0.538682,f1,0.319999,0.057785,29.465172,0.319999,0.057785,29.465172,1,True,9
3,LightGBMXT,0.521284,0.939306,0.892253,0.697023,0.416318,0.553846,f1,0.393499,0.094000,223.723483,0.393499,0.094000,223.723483,1,True,3
4,RandomForestEntr,0.517425,0.932165,0.869887,0.594301,0.458159,0.531507,f1,0.216000,0.055501,29.734565,0.216000,0.055501,29.734565,1,True,6
5,RandomForestGini,0.514758,0.931750,0.866346,0.590786,0.456067,0.527473,f1,0.207059,0.058506,35.627753,0.207059,0.058506,35.627753,1,True,5
6,XGBoost,0.508384,0.939140,0.895889,0.708411,0.396444,0.526984,f1,0.834496,0.172504,168.474053,0.834496,0.172504,168.474053,1,True,11
7,LightGBM,0.504021,0.938559,0.897894,0.701493,0.393305,0.534161,f1,0.233001,0.069504,109.112059,0.233001,0.069504,109.112059,1,True,4
8,CatBoost,0.491643,0.939389,0.904469,0.735417,0.369247,0.495114,f1,0.084000,0.030499,473.556863,0.084000,0.030499,473.556863,1,True,7
9,KNeighborsDist,0.472665,0.923115,0.817541,0.518750,0.434100,0.504065,f1,4.873000,1.029000,1.836931,4.873000,1.029000,1.836931,1,True,2
